In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Multiple_sclerosis"
cohort = "GSE203241"

# Input paths
in_trait_dir = "../../input/GEO/Multiple_sclerosis"
in_cohort_dir = "../../input/GEO/Multiple_sclerosis/GSE203241"

# Output paths
out_data_file = "../../output/preprocess/Multiple_sclerosis/GSE203241.csv"
out_gene_data_file = "../../output/preprocess/Multiple_sclerosis/gene_data/GSE203241.csv"
out_clinical_data_file = "../../output/preprocess/Multiple_sclerosis/clinical_data/GSE203241.csv"
json_path = "../../output/preprocess/Multiple_sclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Age-related blood transcriptional regulators affect disease progression in pediatric multiple sclerosis"
!Series_summary	"The more aggressive clinical disease course of Pediatric Onset Multiple Sclerosis(POMS) as compared to Adult Onset Multiple Sclerosis(AOMS) during the first year disease is supported by higher inflammatory potential promoted by transcriptional level of age-associated genes and transcription factors involved in Cell Cycle, B Cell proliferation and senescent mechanisms."
!Series_summary	"Herein, we compared the blood mononuclear cell transcriptome of POMS and AOMS patients during first year disease. Pediatric Healthy and Adult subjects (PHC, AHC) were used as controls. Correlation analysis of the gene expression with the radiological sign, upstream regulators analysis and clinical assesment were also evaluated."
!Series_overall_design	"A total of 38 MS patients (22 POMS and 16 AOMS) and 21 Healthy controls( 11 PHC and 10 AHC) wer

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Determine if gene expression data is available
# Based on the series title and summary, this dataset appears to contain transcriptome data
# which indicates gene expression data is available
is_gene_available = True

# 2. Variable availability and data type conversion
# From the Sample Characteristics Dictionary, we can see:
# - No direct MS status information (trait)
# - Age information is at key 1
# - Gender information is at key 0

# According to the background information:
# "A total of 38 MS patients (22 POMS and 16 AOMS) and 21 Healthy controls(11 PHC and 10 AHC) were analyzed."
# But we have no way to distinguish these groups in the sample characteristics dictionary

trait_row = None  # MS status not directly identifiable in sample characteristics dictionary
age_row = 1       # Age information is at key 1
gender_row = 0    # Gender information is at key 0

# 2.2 Data Type Conversion Functions
def convert_trait(value: str) -> Optional[int]:
    """
    Convert trait value to binary: 1 for MS, 0 for healthy controls.
    """
    # Since we don't have trait data in the sample characteristics, 
    # this function is defined but won't be used
    return None

def convert_age(value: str) -> Optional[float]:
    """Convert age value to continuous numeric value."""
    if not value or not isinstance(value, str):
        return None
    
    # Extract the age value after the colon
    try:
        if ':' in value:
            age_str = value.split(':', 1)[1].strip()
            return float(age_str)
        return None
    except (ValueError, IndexError):
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender value to binary: 0 for female, 1 for male."""
    if not value or not isinstance(value, str):
        return None
    
    try:
        if ':' in value:
            gender = value.split(':', 1)[1].strip().lower()
            if gender == 'female':
                return 0
            elif gender == 'male':
                return 1
        return None
    except (ValueError, IndexError):
        return None

# 3. Save Metadata
# Since trait_row is None, we're setting is_trait_available to False
is_trait_available = trait_row is not None

# Initial filtering of dataset usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Since trait_row is None, we skip clinical feature extraction
if is_trait_available:
    # This block won't execute because trait_row is None
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical data
    print("\nSelected Clinical Data Preview:")
    print(preview_df(selected_clinical_df))
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("Trait data not available. Skipping clinical feature extraction.")


Trait data not available. Skipping clinical feature extraction.


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers provided in the output
# These identifiers (like '1007_s_at', '1053_at', etc.) appear to be Affymetrix probe IDs
# rather than standard human gene symbols (which would typically be like BRCA1, TP53, etc.)
# Affymetrix probe IDs need to be mapped to gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the key columns in the gene annotation data
# Looking at the gene annotation preview:
# - The 'ID' column contains probe IDs like '1007_s_at', which match gene expression index
# - The 'Gene Symbol' column contains human gene symbols like 'DDR1 /// MIR4640'

# 2. Extract the gene mapping dataframe with the identified columns
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# 4. Display a preview of the resulting gene expression dataframe
print("Mapped gene expression data (first few genes):")
print(gene_data.head())
print(f"Shape of the gene expression data: {gene_data.shape}")


Mapped gene expression data (first few genes):
        GSM6165173  GSM6165174  GSM6165175  GSM6165176  GSM6165177  \
Gene                                                                 
A1CF     -0.045635   -0.102940   -0.063092   -0.080527   -0.073467   
A2M      -0.060192   -0.087774   -0.061199   -0.071824   -0.034100   
A4GALT   -0.090206   -0.101552   -0.100649   -0.083611   -0.119622   
A4GNT    -0.088684   -0.136256   -0.082236   -0.113010   -0.077170   
AAAS      0.332603    0.336113    0.327443    0.253707    0.304259   

        GSM6165178  GSM6165179  GSM6165180  GSM6165181  GSM6165182  ...  \
Gene                                                                ...   
A1CF     -0.028587   -0.088945   -0.098552   -0.054398   -0.094863  ...   
A2M      -0.057952   -0.074955   -0.042982   -0.031651   -0.066185  ...   
A4GALT   -0.160582   -0.122049   -0.154715   -0.136606   -0.139841  ...   
A4GNT    -0.112029   -0.103282   -0.089229   -0.100717   -0.103850  ...   
AAAS      0.

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Original gene expression data shape: {gene_data.shape}")
print(f"Normalized gene expression data shape: {normalized_gene_data.shape}")
print("First 5 normalized gene identifiers:")
print(normalized_gene_data.index[:5])

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Create a simple dataframe with the gene data transposed (samples as rows)
# This gives us a valid dataframe to pass to validate_and_save_cohort_info
sample_df = normalized_gene_data.T
print(f"Sample dataframe shape: {sample_df.shape}")

# Since trait data is not available (as determined in Step 2), 
# the dataset is not usable for our analysis of Multiple Sclerosis
# We set is_biased to False since we're not evaluating the trait distribution
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False,  # We confirmed trait data is not available in Step 2
    is_biased=False,  # We need to provide a value but it's not relevant since trait data is missing
    df=sample_df,  # Using the transposed gene expression data as our dataframe
    note="Dataset contains gene expression data but lacks trait information for Multiple Sclerosis."
)

print("Dataset does not contain the required trait information and thus cannot be used for disease association analysis.")

Original gene expression data shape: (13830, 59)
Normalized gene expression data shape: (13542, 59)
First 5 normalized gene identifiers:
Index(['A1CF', 'A2M', 'A4GALT', 'A4GNT', 'AAAS'], dtype='object', name='Gene')


Normalized gene data saved to ../../output/preprocess/Multiple_sclerosis/gene_data/GSE203241.csv
Sample dataframe shape: (59, 13542)
Dataset does not contain the required trait information and thus cannot be used for disease association analysis.
